# Pepe Image Classification Using Machine Learning

The objective of this notebook is to train a machine learning model and a deep learning model to perform the task of identifying the images of pepe frog. This notebook will also compare the training process and the results of the trained models.

## Data

The data used to train the models in this notebook is present in hugging face. 
Link :- https://huggingface.co/datasets/vikhyatk/synthetic-pepe

## 1. Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from datasets import load_dataset

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import RocCurveDisplay

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

### Importing The Dataset

In [2]:
dataset = load_dataset("vikhyatk/synthetic-pepe")

Resolving data files:   0%|          | 0/505 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
print("done")

done


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image'],
        num_rows: 504
    })
})

In [5]:
dataset[0]

KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['train']"

In [6]:
dataset['image'][0]

KeyError: 'image'